In [106]:
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn import metrics

import numpy as np
import pandas as pd
import re

Innetől a beolvasáshoz segédlet

In [129]:
# One column
# p30_u10 - configuration key: Percentile 30, Uniformity 10
# nature - model type: nature, urban or all
# all - dataset type: nature, urban or all
# 0.6424242854118347 - loss value for picture 0
# 15 more float values for 15 more pictures
df_losses = pd.read_csv('model_losses.csv')
# One column
# p30_u10 - configuration key: Percentile 30, Uniformity 10
# all - dataset type: nature, urban or all
# 0.6424242854118347 - loss value for picture 0
# 15 more float values for 15 more pictures
df_actual = pd.read_csv('image_actual.csv')

In [136]:
for col in df_losses.columns:
    # get first row value for this specific column
    col_name = re.sub('\.\d$', '', col)
    first_row = df_losses.iloc[0][col]
    second_row = df_losses.iloc[1][col]
    new_column_name = f"{col_name}_{first_row}_{second_row}"
    # rename the column with the existing column header plus the first row of that column's data
    df_losses.rename(columns={col: new_column_name}, inplace=True)

# delete first row of data, now that it's merged into the header
df_losses.drop(index=[0,1], inplace=True)
df_losses.drop(columns=df_losses.columns[0], axis=1, inplace=True)
df_losses.reset_index(drop=True, inplace=True)
df_losses=df_losses.astype(float)

for col in df_actual.columns:
    # get first row value for this specific column
    col_name = re.sub('\.\d$', '', col)
    first_row = df_actual.iloc[0][col]
    new_column_name_2 = f"{col_name}_{first_row}"
    # rename the column with the existing column header plus the first row of that column's data
    df_actual.rename(columns={col: new_column_name_2}, inplace=True)

# delete first row of data, now that it's merged into the header
df_actual.drop(index=[0], inplace=True)
df_actual.drop(columns=df_actual.columns[0], axis=1, inplace=True)
df_actual.reset_index(drop=True,inplace=True)
df_actual=df_actual.astype(float)

In [137]:
df_losses_true = pd.DataFrame(df_losses, index=df_losses.index, columns=df_losses.columns)
df_actual_true = pd.DataFrame(df_actual, index=df_actual.index, columns=df_actual.columns)

In [138]:
for col in df_losses_true.columns:
  df_losses_true.loc[df_losses_true[col] > 0.5, col] = 1
  df_losses_true.loc[df_losses_true[col] <= 0.5, col] = 0
for col in df_actual_true.columns:
  df_actual_true.loc[df_actual_true[col] > 0.5, col] = 1
  df_actual_true.loc[df_actual_true[col] <= 0.5, col] = 0

In [ ]:
#A tesztelni kívánt oszlopokat beolvassuk mindkét adaktészletből
loses = np.array(df_losses_true[['p30_u10_nature_urban','p50_u10_nature_all', 'p30_u10_nature_all']])
act = np.array(df_actual_true[['p30_u10_nature_urban','p50_u10_nature_all', 'p30_u10_nature_all']])

Beolvasás és átalakítások vége. Innentől a már korébban meglévő confusion mátrix és score-ok számítása következik.

In [65]:
tn, fp, fn, tp  = metrics.confusion_matrix(act,loses).ravel() #roc_curve(test, pred, drop_intermediate=False)
print(tn, fp, fn, tp)

ValueError: multilabel-indicator is not supported

In [55]:
#Intersection over Union
IoU = tp / (tp+fn+fp)
IoU

1.0

In [56]:
#Sørensen–Dice
DSC = (2*tp) / (2*tp+fn+fp)
DSC

1.0